### The following code is for execution in our production application.  It was initially taken from the 
### SARIMAX_dev1_models and stripped of all of the diagnostic model building code.

In [1]:
# use print only as a function
from __future__ import print_function
import sys
#sys.version_info
#pd.show_versions()

In [2]:
__author__ = 'swe03'

import argparse

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib.pyplot as pplt
from decimal import *

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

from statsmodels.graphics.api import qqplot

from scipy import stats

desired_width = 400
pd.set_option('display.width',desired_width)

In [3]:
#!pip install seaborn

In [4]:
import datetime
from dateutil.relativedelta import relativedelta
#import seaborn as sns
import statsmodels.api as sm  
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [5]:
#!pip install pandasql
from pandasql import PandaSQL 
pdsql = PandaSQL()

##### Read the BQ tables created in the Dst Addr and Port profile 1 or Iterate and Create ADS's 1 programs

In [6]:
#!pip install google-api-python-client
p53_ts = pd.read_gbq("select * from prod.ts_port_53_4wks", project_id="network-sec-analytics")
#p53_ts

Requesting query... ok.
Query running...
Query done.
Processed: 31.5 kb

Retrieving results...
Got 672 rows.

Total time taken 1.19 s.
Finished at 2017-01-26 14:49:59.


In [7]:
p53_ts=p53_ts.sort_values(['date_hour'],ascending=True)

In [8]:
# Create an Hour only variable for the Seasonal Dummies variable creation
p53_ts['hour'] = p53_ts.date_hour.dt.strftime('%H')  # Get the Hour

In [9]:
# Create a Weekday variable for the Seasonal Dummies variable creation
p53_ts['weekday'] = p53_ts.date_hour.dt.strftime('%w')  # Get Weekday

In [10]:
def hour_seasonal_creation(): 
    for index, row in p53_ts.iterrows():
        row_hour = int(row['hour'])
        for i in range(0, 23):
            if row_hour == i:
                p53_ts.loc[index, 'hvar'+ str(i)] = 1
            else:
                p53_ts.loc[index, 'hvar'+ str(i)] = 0
#hour_seasonal_creation()

In [11]:
# Started the week on a Sunday (0)
def weekday_seasonal_creation(): 
    for index, row in p53_ts.iterrows():
        row_weekday = int(row['weekday'])
        for i in range(0, 7):
            if row_weekday == i:
                p53_ts.loc[index, 'wdvar'+ str(i)] = 1
            else:
                p53_ts.loc[index, 'wdvar'+ str(i)] = 0
                
weekday_seasonal_creation()

In [12]:
p53_ts=p53_ts.reset_index(drop=True)

In [13]:
# Need a new Feature created since we repurpose date_hour as Index below
p53_ts['Date_Hour'] = p53_ts['date_hour']
#p53_ts

In [14]:
#Need an Index for the Seasonal Decompostion below
p53_ts.reset_index(inplace=True)
p53_ts = p53_ts.set_index('date_hour')
p53_ts.index.dtype      # Note in the output:dtype('<M8[ns]') but...  np.dtype('datetime64[ns]') = np.dtype('<M8[ns]'):

dtype('<M8[ns]')

##### Execute some Univariate Statistics

In [15]:
# Remove the Outliers
p53_ts[p53_ts['bytes_mean'] >1000] = 150
p53_ts[p53_ts['count'] > 75] = 50
p53_ts[p53_ts['count'] < 35] = 50
#p53_ts['count'].describe()

In [16]:
##exog = np.column_stack(list_of_lists)
a = np.array(p53_ts['wdvar0'])  # This is Sunday
b = np.array(p53_ts['wdvar1'])
c = np.array(p53_ts['wdvar2'])
d = np.array(p53_ts['wdvar3'])
e = np.array(p53_ts['wdvar4'])
f = np.array(p53_ts['wdvar5'])
g = np.array(p53_ts['wdvar6'])  # Saturday.  This is the Reference day (i.e., Intercept) for Exog 2
exog1 = np.column_stack((a,b,c,d,e,f,g))
exog2 = np.column_stack((a,b,c,d,e,f))  # This is for the initial modeling using the exog parameter  

# This set is for the subsequent one step ahead predictions for day of week dummies. In this case, 2 days.
d0 = np.array([0,1])                  # Need (2,6).  This is Row, Column
d1 = np.array([0,0])
d2 = np.array([0,0])                         
d3 = np.array([0,0])
d4 = np.array([0,0])
d5 = np.array([0,0])   
exog3 = np.column_stack((d0,d1,d2,d3,d4,d5))                    


#bytes_mean exog congif.  
# Need N Rows(Nbr of one step ahead Hours), 1 Col
exog4 = np.empty((2,1)) 
exog4[0] = 122.25
exog4[1] = 125.50
#exog4


In [17]:
mod = sm.tsa.statespace.SARIMAX(p53_ts['count'], 
     trend='n', order=(5,0,1), seasonal_order=(6,0,0,24))
results = mod.fit()
#print(results.summary())                                

/Users/swe03/anaconda/lib/python3.5/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)
/Users/swe03/anaconda/lib/python3.5/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


In [18]:
print(results.summary())

                                 Statespace Model Results                                 
Dep. Variable:                              count   No. Observations:                  672
Model:             SARIMAX(5, 0, 1)x(6, 0, 0, 24)   Log Likelihood               -2127.399
Date:                            Thu, 26 Jan 2017   AIC                           4280.797
Time:                                    14:52:15   BIC                           4339.431
Sample:                                11-20-2016   HQIC                          4303.505
                                     - 12-17-2016                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          1.1116      0.043     25.669      0.000       1.027       1.196
ar.L2         -0.0011      0.058   

In [19]:
# In-sample one-step-ahead predictions
# NOTE:  start is 672 but since this is a 0 based index the last history record is 671 for 672 total records and 
#        the start value is actually the first forecasted record and index value.
# Start and End time will be passed into the function dynamically as we progress through time. 
# These Start and End times will cycle through a fixed set of index values, assuming we have always keep the time 
# frame fixed (i.e., one year)
predict = results.get_prediction(start=672, end=673)
           #,exog = exog4  ) # Bytes_Mean for two Hours
           #exog = exog2    # Day of Week  
predict_ci = predict.conf_int()

In [20]:
predict.predicted_mean

2016-12-18 00:00:00    51.418903
2016-12-18 01:00:00    50.939035
Freq: H, dtype: float64

In [21]:
predict_ci.sort_index(ascending=True)

,lower count,upper count
2016-12-18 00:00:00,40.303764,62.534043
2016-12-18 01:00:00,39.674788,62.203282
